## 6. 배포

- UI/UX 고려
- 프로토 타입 형태 배포: OpenCV 프레임, Gradio, Streamlit 


#### 6-1. OpenCV 프레임


In [1]:
import openvino as ov

core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

In [3]:
import openvino as ov
import cv2
import numpy as np
from pathlib import Path

# Load the model and prepare the input/output layers
core = ov.Core()
model = core.read_model(model='models/v3-small_224_1.0_float.xml')
compiled_model = core.compile_model(model=model, device_name="CPU")
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

# Load ImageNet classes
imagenet_filename = Path('data/imagenet_2012.txt')
imagenet_classes = imagenet_filename.read_text().splitlines()
imagenet_classes = ["background"] + imagenet_classes  # Include the background class

def preprocess(image):
    # Resize the image to 224x224 and add a batch dimension
    input_image = cv2.resize(src=image, dsize=(224, 224))
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension
    return input_image 

def predict_image(image_path, background_image_path):
    # Read the input image
    image = cv2.imread(image_path)
    input_image = preprocess(image)  # Preprocess the image
    result_infer = compiled_model([input_image])[output_layer]  # Perform inference
    result_index = np.argmax(result_infer)  # Get the index of the highest score
    
    # Get the predicted class name
    predicted_class = imagenet_classes[result_index]

    # Read the background image
    bg = cv2.imread(background_image_path)

    # Resize the input image to match the proper position of background image
    image_h, image_w = image.shape[0], image.shape[1]
    new_h = 500
    new_w = int((new_h/image_h)*image_w)
    image_resize = cv2.resize(image, (new_w, new_h))

    
    xmax = bg.shape[1] - 200
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h
    
    # Overlay the input image on the background image
    bg[ymin:ymax, xmin:xmax] = image_resize
    
    # Overlay the predicted class label on the sample image
    cv2.putText(bg, predicted_class, (600, 580), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Display the final combined image
    cv2.imshow("Sample Image with Prediction on Background", bg)

    # Wait for a key press and close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
sample_image_path = "./data/coco.jpg"  # Replace with your actual image file path
background_image_path = "./data/background.jpg"  # Replace with your actual background image file path

predict_image(sample_image_path, background_image_path)


#### 6-2. Gradio
https://www.gradio.app/guides/quickstart

In [8]:
import openvino as ov

core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

In [9]:
import openvino as ov
import cv2
import numpy as np
import PIL
from pathlib import Path
import gradio as gr

# Load the model and prepare the input/output layers
core = ov.Core()
model = core.read_model(model='models/v3-small_224_1.0_float.xml')
compiled_model = core.compile_model(model=model, device_name="CPU")
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

# Load ImageNet classes
imagenet_filename = Path('data/imagenet_2012.txt')
imagenet_classes = imagenet_filename.read_text().splitlines()
imagenet_classes = ["background"] + imagenet_classes  # Include the background class

def preprocess(image):
    # Convert the PIL image to a NumPy array and resize to 224x224
    image = np.array(image)  # Convert PIL image to numpy array
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR for OpenCV
    input_image = cv2.resize(src=image, dsize=(224, 224))
    input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension
    return input_image 

def predict_image(image):
    input_image = preprocess(image)  # Preprocess the image
    result_infer = compiled_model([input_image])[output_layer]  # Perform inference
    result_index = np.argmax(result_infer)  # Get the index of the highest score
    
    # Get the predicted class name
    predicted_class = imagenet_classes[result_index]
    return predicted_class  # Return just the predicted class name

# Set up the Gradio interface
gr.Interface(fn=predict_image,
             inputs=gr.Image(type="pil"),
             outputs=gr.Textbox(),  # Use Textbox to show the predicted class
             examples=["./data/coco.jpg"]).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


#### 6-3. Streamlit
https://docs.streamlit.io/get-started/tutorials/create-an-app#create-your-first-app

In [6]:
import openvino as ov

core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

In [7]:
import streamlit as st
st.title("Hello Image Classification :sun_with_face:")

2024-10-01 13:22:07.909 
  command:

    streamlit run C:\Users\kojan\.conda\envs\BrainAI\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [ ]:
#!jupyter nbconvert --to script hello-world-AI-Project-Streamlit.ipynb